In [1]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [2]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
print(game)
print(game.board_value())

+------+------+------+------+
|      |      |  2   |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
4


In [3]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

The below code is the base 

In [4]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [8]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

KeyboardInterrupt: 

In [19]:
f = open("res.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

In [5]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [6]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(6)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+------+------+
|  4   | 1024 |  2   |  8   |  32  |  4   |
+------+------+------+------+------+------+
|      |  2   | 2048 |  2   | 128  |  32  |
+------+------+------+------+------+------+
|      |  4   |  8   | 256  |  8   |  16  |
+------+------+------+------+------+------+
|      |      | 512  |  4   |  2   |  4   |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  4   |
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
1
5589556.0
137896996573.75082
1
+------+------+------+------+------+------+
|  16  |  2   | 512  |  2   | 2048 | 1024 |
+------+------+------+------+------+------+
|  2   |  4   | 1024 |  16  |  16  |  4   |
+------+------+------+------+------+------+
|  4   | 256  |  2   | 256  | 512  |  8   |
+------+------+------+------+------+------+
|      |  32  |  16  | 128  |  2   |  16  |
+------+------+------+------+------+-----

14
+------+------+------+------+------+------+
|      |      |      |  8   |  2   |  16  |
+------+------+------+------+------+------+
| 256  |  32  |  8   |  64  |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  8   |  64  |  32  | 1024 |  4   |
+------+------+------+------+------+------+
|      |  4   | 128  |  4   |  8   |  2   |
+------+------+------+------+------+------+
|      |      | 512  |      | 512  | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  4   |
+------+------+------+------+------+------+
1
5860180.0
68097287077.16129
15
+------+------+------+------+------+------+
|  4   | 1024 |  2   |  8   |  2   | 128  |
+------+------+------+------+------+------+
|  2   |  16  | 128  |  32  | 256  |  2   |
+------+------+------+------+------+------+
|      |  2   |  64  |  4   |  8   |  32  |
+------+------+------+------+------+------+
|      |  2   |  4   |  8   | 256  |  8   |
+------+------+------+------+------+----

28
+------+------+------+------+------+------+
|  4   |  16  | 1024 |  16  |  2   |      |
+------+------+------+------+------+------+
|  4   |  64  |  8   |  32  |  8   |  4   |
+------+------+------+------+------+------+
|  8   |  4   |  32  |  4   |  32  |      |
+------+------+------+------+------+------+
|  4   |  8   |      |      |      |      |
+------+------+------+------+------+------+
| 2048 | 1024 |      | 1024 |      |      |
+------+------+------+------+------+------+
|  4   |      |  4   |      |  4   |      |
+------+------+------+------+------+------+
1
7348116.0
64692429535.7125
29
+------+------+------+------+------+------+
|      |      |      |  2   | 1024 | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  16  |  8   |
+------+------+------+------+------+------+
|      |  4   |  32  |  64  |  4   |  2   |
+------+------+------+------+------+------+
|      |      |      |  64  |  2   |  8   |
+------+------+------+------+------+-----

42
+------+------+------+------+------+------+
|  4   |  16  |  8   |  4   |      |      |
+------+------+------+------+------+------+
|  2   |  4   | 256  |  4   |  8   |      |
+------+------+------+------+------+------+
|  2   |  4   |  64  |  4   |  2   |      |
+------+------+------+------+------+------+
|  2   | 2048 |      |      |      |      |
+------+------+------+------+------+------+
|  4   |  8   |      |      |      |      |
+------+------+------+------+------+------+
|  8   |  32  |      |      |      |      |
+------+------+------+------+------+------+
1
4265600.0
80014329922.8656
43
+------+------+------+------+------+------+
|      |      |  2   |  8   |  16  |  32  |
+------+------+------+------+------+------+
|      |      |      |      | 2048 |  2   |
+------+------+------+------+------+------+
|      |  2   |  4   |  8   |  32  |  8   |
+------+------+------+------+------+------+
|      |      |      |  32  |  8   |  2   |
+------+------+------+------+------+-----

56
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
|  2   |  16  |      |      |      |      |
+------+------+------+------+------+------+
|  8   |  8   |  4   |      |      |      |
+------+------+------+------+------+------+
|  4   |  32  |  8   |  2   |  8   |  2   |
+------+------+------+------+------+------+
|  4   | 2048 |  16  |  4   |      |      |
+------+------+------+------+------+------+
|  16  |  2   |  8   |  32  |  8   |  2   |
+------+------+------+------+------+------+
1
4197588.0
69799742814.45586
57
+------+------+------+------+------+------+
|  2   | 512  |  8   |  4   | 2048 | 256  |
+------+------+------+------+------+------+
|  32  | 256  |  2   |  8   |  32  |  4   |
+------+------+------+------+------+------+
|  16  |  2   |  32  |  4   |  8   |      |
+------+------+------+------+------+------+
|  64  |  32  | 1024 |  16  |  4   |      |
+------+------+------+------+------+----

70
+------+------+------+------+------+------+
|      |  8   |  2   |  8   |  4   |  8   |
+------+------+------+------+------+------+
|      |      |  16  |  8   |  16  | 2048 |
+------+------+------+------+------+------+
|      |      |  8   |  32  |  2   |  8   |
+------+------+------+------+------+------+
|      |      |      |  64  | 128  | 2048 |
+------+------+------+------+------+------+
|      | 1024 |      | 1024 |  32  |  4   |
+------+------+------+------+------+------+
|      |      |      | 1024 |  2   | 1024 |
+------+------+------+------+------+------+
1
12606380.0
125980036204.67894
71
+------+------+------+------+------+------+
|  2   | 256  |  8   |  32  | 2048 |  16  |
+------+------+------+------+------+------+
|  16  | 512  |  16  |  8   | 1024 |  8   |
+------+------+------+------+------+------+
|  32  | 256  |  8   | 512  |      |      |
+------+------+------+------+------+------+
|  8   |  32  |      |  2   | 1024 |  8   |
+------+------+------+------+------+--

84
+------+------+------+------+------+------+
|  4   |  64  |  8   |  4   |  2   |      |
+------+------+------+------+------+------+
|  16  |  8   |  16  |  2   |  32  |      |
+------+------+------+------+------+------+
|  2   |  4   |  2   |  8   | 1024 |      |
+------+------+------+------+------+------+
| 256  | 128  | 256  |  4   |      |      |
+------+------+------+------+------+------+
|  2   |  4   | 2048 |      |      |      |
+------+------+------+------+------+------+
|  4   |  4   |  4   |      |      |      |
+------+------+------+------+------+------+
1
5396308.0
120872748642.88412
85
+------+------+------+------+------+------+
|      |      | 1024 |      | 1024 | 2048 |
+------+------+------+------+------+------+
|      |      |      |      |  2   |  16  |
+------+------+------+------+------+------+
|      |      |      |  16  | 512  |  8   |
+------+------+------+------+------+------+
|      |      |  4   | 512  |  64  |  4   |
+------+------+------+------+------+---

98
+------+------+------+------+------+------+
|  8   |  4   |  2   |  32  |  4   |  2   |
+------+------+------+------+------+------+
|  4   |  64  | 128  | 512  |  16  |  8   |
+------+------+------+------+------+------+
|      |      |  2   |  16  |  4   |  32  |
+------+------+------+------+------+------+
|      |      |      | 2048 | 512  |  2   |
+------+------+------+------+------+------+
|      |      |      |  2   |  4   |  2   |
+------+------+------+------+------+------+
|      |      |      |  2   |  4   |  4   |
+------+------+------+------+------+------+
1
4741900.0
202589559611.77078
99
+------+------+------+------+------+------+
|  2   |      |      |      |      |      |
+------+------+------+------+------+------+
|  64  |      |      |      |      |      |
+------+------+------+------+------+------+
|  4   |  8   |      |      | 1024 |  32  |
+------+------+------+------+------+------+
|  8   | 256  |  16  |      |      |      |
+------+------+------+------+------+---

In [7]:
f = open("res_boot3.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 3 - Random Walk

In [14]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(6)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            board.playRandUser()
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

2048.0
+------+------+------+------+------+------+
|  2   | 512  |  16  |  8   |  4   |  2   |
+------+------+------+------+------+------+
|  4   |  2   | 128  |  4   |  16  |  8   |
+------+------+------+------+------+------+
|  16  | 256  | 1024 |  8   |  4   |  2   |
+------+------+------+------+------+------+
|  2   |  32  | 2048 |  4   |  8   |  16  |
+------+------+------+------+------+------+
|  8   |  4   |  8   |  64  | 128  |  8   |
+------+------+------+------+------+------+
|  4   |  8   |  2   |  8   |  16  |  2   |
+------+------+------+------+------+------+
2
5610444.0
0
1024.0
+------+------+------+------+------+------+
|  8   | 256  |  8   |  32  |  2   |  4   |
+------+------+------+------+------+------+
|  32  |  16  | 128  | 1024 |  4   |  8   |
+------+------+------+------+------+------+
|  8   |  2   |  8   | 512  |  32  |  2   |
+------+------+------+------+------+------+
|  64  |  16  | 512  |  64  | 256  |  4   |
+------+------+------+------+------+------+
|  4

2048.0
+------+------+------+------+------+------+
|  4   |  8   |  4   | 2048 |  4   |  2   |
+------+------+------+------+------+------+
|  2   | 2048 |  16  | 128  |  16  |  8   |
+------+------+------+------+------+------+
|  8   |  32  |  64  |  4   | 256  |  64  |
+------+------+------+------+------+------+
|  4   | 1024 | 128  |  2   |  4   |  16  |
+------+------+------+------+------+------+
|  2   |  16  |  8   |  4   | 512  |  2   |
+------+------+------+------+------+------+
|  8   |  32  |  4   |  8   |  2   |  32  |
+------+------+------+------+------+------+
2
9810456.0
0
2048.0
+------+------+------+------+------+------+
|  32  |  4   |  2   |  16  |  8   |  2   |
+------+------+------+------+------+------+
|  4   |  16  |  32  |  4   |  2   | 512  |
+------+------+------+------+------+------+
|  2   | 512  | 2048 |  16  | 256  |  4   |
+------+------+------+------+------+------+
| 128  |  4   |  64  |  2   |  32  | 512  |
+------+------+------+------+------+------+
|  4

4096.0
+------+------+------+------+------+------+
|  2   |  8   |  4   |  2   | 256  |  2   |
+------+------+------+------+------+------+
|  64  |  32  | 4096 |  16  |  8   |  4   |
+------+------+------+------+------+------+
|  2   |  4   | 256  |  8   |  2   | 128  |
+------+------+------+------+------+------+
|  4   |  16  |  4   | 128  |  4   | 2048 |
+------+------+------+------+------+------+
|  2   | 512  |  64  |  16  |  8   |  2   |
+------+------+------+------+------+------+
|  8   |  32  |  2   |  8   |  4   |  8   |
+------+------+------+------+------+------+
2
21409104.0
0
1024.0
+------+------+------+------+------+------+
|  2   |  4   |  8   |  16  |  8   |  2   |
+------+------+------+------+------+------+
|  32  |  2   |  32  |  8   | 128  |  4   |
+------+------+------+------+------+------+
|  2   | 256  |  4   |  16  | 512  |  32  |
+------+------+------+------+------+------+
|  8   |  4   | 1024 | 256  |  4   |  8   |
+------+------+------+------+------+------+
| 5

4096.0
+------+------+------+------+------+------+
|  4   |  32  |  2   |  8   |  4   |  2   |
+------+------+------+------+------+------+
|  8   |  64  |  4   |  32  |  16  |  32  |
+------+------+------+------+------+------+
|  4   |  16  |  32  | 512  | 128  |  2   |
+------+------+------+------+------+------+
|  2   | 256  |  2   | 4096 | 2048 | 1024 |
+------+------+------+------+------+------+
|  4   |  64  |  8   |  2   |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  16  |  2   |  8   |  2   |  8   |
+------+------+------+------+------+------+
2
22377912.0
0
1024.0
+------+------+------+------+------+------+
|  2   |  4   | 128  |  8   |  16  |  2   |
+------+------+------+------+------+------+
|  8   | 512  |  16  |  64  |  8   |  4   |
+------+------+------+------+------+------+
|  2   |  32  |  64  | 512  | 256  |  32  |
+------+------+------+------+------+------+
|  8   |  4   | 256  |  8   |  64  |  2   |
+------+------+------+------+------+------+
|  

2048.0
+------+------+------+------+------+------+
|  2   |  4   |  8   |  64  |  8   |  2   |
+------+------+------+------+------+------+
|  8   |  16  |  4   | 128  |  16  |  8   |
+------+------+------+------+------+------+
|  2   |  64  | 2048 |  16  | 512  |  2   |
+------+------+------+------+------+------+
|  16  | 512  |  32  |  2   |  16  |  4   |
+------+------+------+------+------+------+
|  4   |  64  |  4   |  16  | 128  |  16  |
+------+------+------+------+------+------+
|  2   |  16  |  32  |  2   |  4   |  2   |
+------+------+------+------+------+------+
2
4768128.0
0
2048.0
+------+------+------+------+------+------+
|  4   |  16  |  2   |  4   |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  4   | 2048 |  32  | 512  |  8   |
+------+------+------+------+------+------+
|  4   |  8   | 128  | 1024 |  32  |  2   |
+------+------+------+------+------+------+
|  2   |  16  |  4   |  16  | 128  |  16  |
+------+------+------+------+------+------+
|  1

16384.0
+------+------+------+------+------+------+
|  2   |  16  |  4   |  2   |  4   |  2   |
+------+------+------+------+------+------+
|  4   |  8   |  2   |16384 | 128  |  4   |
+------+------+------+------+------+------+
| 256  |  64  |  32  | 1024 |  4   |  8   |
+------+------+------+------+------+------+
|  2   |  8   |  16  |  8   | 128  |  4   |
+------+------+------+------+------+------+
|  8   |  64  | 2048 |  64  | 256  |  2   |
+------+------+------+------+------+------+
|  2   |  8   |  32  |  2   |  4   |  8   |
+------+------+------+------+------+------+
2
273857616.0
0
2048.0
+------+------+------+------+------+------+
|  4   |  2   |  8   |  2   |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  8   |  16  |  32  |  2   |  16  |
+------+------+------+------+------+------+
| 1024 |  16  | 2048 | 1024 |  32  |  4   |
+------+------+------+------+------+------+
|  4   |  2   |  8   | 512  | 256  |  16  |
+------+------+------+------+------+------+
|

2048.0
+------+------+------+------+------+------+
|  8   | 128  |  2   | 512  |  4   |  2   |
+------+------+------+------+------+------+
|  16  |  2   |  32  |  4   |  32  |  8   |
+------+------+------+------+------+------+
|  4   | 128  |  16  | 1024 |  16  |  2   |
+------+------+------+------+------+------+
|  2   |  32  |  2   | 2048 |  4   |  16  |
+------+------+------+------+------+------+
|  4   |  16  |  4   |  8   | 128  |  4   |
+------+------+------+------+------+------+
|  2   |  8   |  32  |  64  |  8   |  2   |
+------+------+------+------+------+------+
2
5564112.0
0
1024.0
+------+------+------+------+------+------+
|  2   |  4   |  16  |  8   |  2   |  4   |
+------+------+------+------+------+------+
|  32  |  16  |  8   |  16  |  8   |  2   |
+------+------+------+------+------+------+
|  8   | 512  |  32  |  64  | 256  |  4   |
+------+------+------+------+------+------+
|  4   |  32  | 256  |  16  |  64  |  32  |
+------+------+------+------+------+------+
|  8

4096.0
+------+------+------+------+------+------+
|  16  |  8   |  16  |  32  |  4   |  2   |
+------+------+------+------+------+------+
|  8   |  2   | 256  | 128  | 2048 |  4   |
+------+------+------+------+------+------+
|  2   |  32  | 1024 |  16  |  64  |  8   |
+------+------+------+------+------+------+
|  16  | 128  | 4096 |  4   | 128  |  16  |
+------+------+------+------+------+------+
|  4   |  32  |  8   |  16  | 256  |  8   |
+------+------+------+------+------+------+
|  2   |  8   | 512  |  2   |  8   |  4   |
+------+------+------+------+------+------+
2
22471716.0
0
8192.0
+------+------+------+------+------+------+
|  2   |  8   |  32  | 256  |  4   |  8   |
+------+------+------+------+------+------+
|  4   | 128  |  64  |  4   | 256  |  2   |
+------+------+------+------+------+------+
|  8   |  2   | 8192 |  64  |  4   |  64  |
+------+------+------+------+------+------+
|  32  | 512  | 1024 |  2   |  64  |  4   |
+------+------+------+------+------+------+
|  

In [15]:
f = open("res_rand6-score.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 4 - How High n-steps

In [16]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
                
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        #print(self.unexplored_moves)
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        #print(leaf.grid)
        sim_res = rollout_o(leaf, start_grid)
        #print(sim_res)
        backpropagate(leaf, sim_res)
        #print(root.children)

    # When done iterating, return the 'best' child of the root node.\
    try:
        return best_child(root).move
    except:
        #print(root.children)
        return root

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        #print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)
    
    # If the node is terminal, return it
    if node.is_terminal_state:
        #print(1)
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    return sample.board_value()/(16384 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [18]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(6)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board,100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

2097152.0
+------+------+------+------+------+------+
|  4   |  32  |  2   |  32  |  8   |  16  |
+------+------+------+------+------+------+
|  2   |  64  |  16  |131072|2097152|  8   |
+------+------+------+------+------+------+
|  8   |262144|65536 |  2   |  16  |  2   |
+------+------+------+------+------+------+
|  16  |  32  | 512  |32768 |  4   |1048576|
+------+------+------+------+------+------+
|  4   | 2048 |131072|  32  |65536 |  2   |
+------+------+------+------+------+------+
|  2   |  64  |  4   |  8   |  16  |  4   |
+------+------+------+------+------+------+
2
5610305500776.0
321761359610.85675


KeyboardInterrupt: 

# Addtion testing for 4x4 with bootstraping how high

In [5]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
                
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        #print(self.unexplored_moves)
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        #print(leaf.grid)
        sim_res = rollout_o(leaf, start_grid)
        #print(sim_res)
        backpropagate(leaf, sim_res)
        #print(root.children)

    # When done iterating, return the 'best' child of the root node.\
    try:
        return best_child(root).move
    except:
        #print(root.children)
        return root

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        #print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)
    
    # If the node is terminal, return it
    if node.is_terminal_state:
        #print(1)
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    return sample.board_value()/(16384 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [ ]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

1024.0
+------+------+------+------+
|  2   |  4   | 128  |  4   |
+------+------+------+------+
|  16  |  2   |  8   |  2   |
+------+------+------+------+
|  2   |  8   |  16  |  8   |
+------+------+------+------+
|  4   |  32  | 1024 |  2   |
+------+------+------+------+
2
1066756.0
177054002084.64938
512.0
+------+------+------+------+
|  2   |  4   |  8   |  2   |
+------+------+------+------+
|  16  |  8   |  32  |  8   |
+------+------+------+------+
|  4   | 128  |  64  | 512  |
+------+------+------+------+
|  2   |  16  |  4   |  2   |
+------+------+------+------+
2
284416.0
318356803925.7135
